In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
import openaq
api = openaq.OpenAQ()

In [12]:
from shapely.geometry import Polygon, Point
import folium
import geopandas as gpd

In [5]:
import requests

In [8]:
from h3 import h3

In [6]:
from datetime import datetime, timedelta
dt_past_24_hours = (datetime.now() - timedelta(hours = 24)).strftime("%Y-%m-%d %H:%M:%S")
dt_past_24_hours

'2022-10-07 10:28:45'

In [22]:
#get pollution data of last 24 hours for delhi
pollution_data = api.measurements(city='Delhi', df=True, limit=1000)
pollution_data = pollution_data.loc[(datetime.now() - timedelta(hours = 24)).strftime("%Y-%m-%d %H:%M:%S"):]
pollution_data_24_hrs = pollution_data.groupby(by=['location', 'parameter', 'coordinates.latitude','coordinates.longitude']).mean('value').reset_index()
pollution_data_24_hrs['point'] = pollution_data_24_hrs.apply(lambda x: Point(x['coordinates.longitude'],
                                                                            x['coordinates.latitude']), axis=1)
pollution_data_24_hrs

,location,parameter,coordinates.latitude,coordinates.longitude,value,point
0,"Alipur, Delhi - DPCC",co,28.815329,77.153010,625.000,POINT (77.15301 28.815329)
1,"Alipur, Delhi - DPCC",no2,28.815329,77.153010,5.325,POINT (77.15301 28.815329)
2,"Alipur, Delhi - DPCC",o3,28.815329,77.153010,22.550,POINT (77.15301 28.815329)
3,"Alipur, Delhi - DPCC",pm10,28.815329,77.153010,28.250,POINT (77.15301 28.815329)
4,"Alipur, Delhi - DPCC",pm25,28.815329,77.153010,8.750,POINT (77.15301 28.815329)
...,...,...,...,...,...,...
175,"Wazirpur, Delhi - DPCC",no2,28.699793,77.165453,11.450,POINT (77.165453 28.699793)
176,"Wazirpur, Delhi - DPCC",o3,28.699793,77.165453,14.050,POINT (77.165453 28.699793)
177,"Wazirpur, Delhi - DPCC",pm10,28.699793,77.165453,36.000,POINT (77.165453 28.699793)
178,"Wazirpur, Delhi - DPCC",pm25,28.699793,77.165453,20.000,POINT (77.165453 28.699793)


In [63]:
#creating h3 cells in delhi
## Lets get boundary of Kolkata city from OSM and plot it
search_string = 'Delhi'
urlData = requests.get(f'https://nominatim.openstreetmap.org/search.php?q={search_string}&polygon_geojson=1&format=json').json()
bounds = pd.DataFrame.from_dict(urlData)
delhib = gpd.GeoDataFrame({'city':'Delhi (India)','geometry' : 
                           Polygon(bounds[(bounds.type == 'administrative') & (bounds.display_name == 'Delhi, India')].geojson.to_dict()[1]['coordinates'][0])}, index=[0])
hexs = h3.polyfill(
    Polygon(bounds[bounds.type == 'administrative'].head(2).geojson.to_dict()[1]['coordinates'][0]).__geo_interface__, 8, geo_json_conformant = True)
                                                                                                                                            
polygonise = lambda hex_id: Polygon(
    h3.h3_to_geo_boundary(
    hex_id, geo_json=True))
all_polys = gpd.GeoSeries(list(map(polygonise, hexs)), \
                          index=hexs, \
                          crs="EPSG:4326")

gdf_delhi_h3 = gpd.GeoDataFrame(all_polys, columns=['geometry'])
gdf_delhi_h3.explore()
bounds_folium = []
for pt in bounds[(bounds.type == 'administrative') & (bounds.display_name == 'Delhi, India')].geojson.to_dict()[1]['coordinates'][0]:
    bounds_folium.append([pt[1],pt[0]])

m = gdf_delhi_h3.explore()
folium.PolyLine(bounds_folium, color='black').add_to(m)
m                                                                                                                                                                                                                                                             

# To read more about IDW
[idw-interpolation](https://gisgeography.com/inverse-distance-weighting-idw-interpolation/)

In [105]:
def get_predicted_pollution(hexagone, pollution_data, closest_station_cutoff, p):
    #finding if a pollution stations lies in the hexagone, return mean of those stations
    inside_station = []
    flag = 0
    for i , r in pollution_data.iterrows():
        if hexagone.contains(r['point']):
            inside_station.append(r['value'])
            flag = 1
    if flag == 1:
        return (round((sum(inside_station) / len(inside_station)), 2))
    
    
    # implement idw 
    centroid = hexagone.centroid
    
    distances = pollution_data['point'].apply(lambda x: centroid.distance(x))
    distances.index = pollution_data['location']
    distances = distances.sort_values(ascending=True)
    distances = distances.head(closest_station_cutoff)
    num = 0
    den = 0
    for i, v in distances.items():
        num = num + ((pollution_data[pollution_data['location'] == i]['value'].values[0])/(v*100)**p)
        den = den + (1/(v*100)**p)    
    return (round((num / den), 2))
    

In [113]:
pollution_data = pollution_data_24_hrs[pollution_data_24_hrs.parameter == 'pm25']
gdf_delhi_h31 = gdf_delhi_h3.copy()
gdf_delhi_h31['predicted_pm25_ugm3'] = gdf_delhi_h31['geometry'].apply(lambda hexs: get_predicted_pollution(hexs, pollution_data, 5, 2))
gdf_delhi_h31.head()

,geometry,predicted_pm25_ugm3
883da1ab29fffff,"POLYGON ((77.30734 28.66879, 77.30772 28.66403, 77.31283 28.66202, 77.31756 28.66477, 77.31718 28.66952, 77.31207 28.67153, 77.30734 28.66879))",9.51
883da18905fffff,"POLYGON ((76.86843 28.59220, 76.86884 28.58745, 76.87396 28.58546, 76.87867 28.58822, 76.87826 28.59297, 76.87314 28.59496, 76.86843 28.59220))",12.11
883da184a7fffff,"POLYGON ((77.16423 28.84485, 77.16462 28.84009, 77.16975 28.83807, 77.17449 28.84083, 77.17409 28.84560, 77.16897 28.84761, 77.16423 28.84485))",9.92
883da18e05fffff,"POLYGON ((77.02783 28.75538, 77.02824 28.75062, 77.03336 28.74861, 77.03809 28.75137, 77.03769 28.75613, 77.03256 28.75814, 77.02783 28.75538))",13.76
883da10217fffff,"POLYGON ((77.18565 28.52336, 77.18604 28.51861, 77.19114 28.51661, 77.19585 28.51935, 77.19546 28.52410, 77.19036 28.52610, 77.18565 28.52336))",5.99


In [118]:
#plot the hyperlocal air pollution map
m = gdf_delhi_h31.explore(column='predicted_pm25_ugm3')
m